# Práctica 1: Análisis exploratorio de datos, preprocesamiento y validación de modelos de clasificación

### Minería de Datos: Curso académico 2020-2021


* Mª Mercedes Guijarro

En esta práctica se realizará un informe reproduciendo el estudio hecho con iris en el cuaderno de la Práctica 1, por lo que para ello trabajaremos algunos de los aspectos más importantes del proceso *KDD* :

* Almacenamiento y carga de datos
* Análisis exploratorio de datos
* Preprocesamiento de datos
* Validación de modelos de clasificación


El objetivo de la práctica será aprender a cargar, explorar y preparar nuestros datos, aprender y validar distintos modelos de clasificación y ser capaces de interpretar los resultados obtenidos.

# 1. Preliminares

Antes de comenzar es necesario cargar las librerías a emplear para que estén disponibles para su posterior uso:

In [ ]:
# Third party
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.tree import DecisionTreeClassifier

# Local application
import miner_a_de_datos_an_lisis_exploratorio_utilidad as utils

import pandas as pd
import numpy as np

Además, fijamos una semilla para que los experimentos sean reproducibles:

In [ ]:
seed = 27912

 # 2. Carga de datos Diabetes dataset
## 1. Diabetes dataset
     Para comenzar tenemos que hacer la carga del dataset, el cual almacena información sobre ciertas variables predictoras (nº de embarazos, presión sanguínea, glucosa...) para predecir si una persona tiene o no diabetes.

Comenzamos cargando el conjunto de datos `diabetes`:

In [ ]:
filepath = "../input/pima-indians-diabetes-database/diabetes.csv"


data = pd.read_csv(filepath, dtype={"Outcome": 'category'})

En este caso no tenemos ninguna variable que corresponda con un identificador de casos el conjunto de datos. 
Indicamos la variable clase que en este caso es `Outcome`

Una vez realizada la carga el conjuntod de datos, pasamos a comprobar que se ha realizado correctamente, y que las variables y valores están dentro de lo esperado. 
Para ello podemos usar la función `head` para obtener las `n` primeras instancias del conjunto de datos:


In [ ]:
data.head(5)

Y también podemos utilizar `info` para, además de comprobar que se haya cargado correctamente, ver las columnas (variables) de nuestro conjunto:

In [ ]:
data.info()

Aunque hayamos comprobado que se ha cargado correctamente además podemos realizar una muestra aleatoria, ya que la anterior es una muestra sesgada. 

In [ ]:
data.sample(5, random_state=seed)

Una vez hemos realizado las comprobaciones sobre la carga del conjunto de datos, pasamos separar nuestro conjunto en dos subconjuntos, uno para las variables predictoras (`X`) y otro con las variables objetivos (`Y`).


In [ ]:
(X, y) = utils.divide_dataset(data, target="Outcome")

Y comprobamos mediante una muestra aleatoria que se haya realizado la separación correctamente:

In [ ]:
X.sample(5, random_state=seed)

In [ ]:
y.sample(5, random_state=seed)

A continuación pasaremos a separar nuestro conjunto de datos en dos (proceso *holdout*) :

* Una muestra de entrenamiento (70%)
* Una muestra de prueba (30%)

Realizamos un holdout:

In [ ]:
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)

Por defecto se aleatorizan las instancias del conjunto  antes de realizar el *holdout* (`shuffle=True`).

Comprobamos que el conjunto de datos se ha dividido correctamente en training y test:

In [ ]:
X_train.sample(5, random_state=seed)

In [ ]:
X_test.sample(5, random_state=seed)

Por último, finalizamos con la variable objetivo del conjunto de datos de entrenamiento:

In [ ]:
y_train.sample(5, random_state=seed)

Y test:

In [ ]:
y_test.sample(5, random_state=seed)

Para facilitar el análisis exploratorio de datos, volvemos a juntar las variables predictoras con la variable clase. Comenzamos con el conjunto de datos de entrenamiento:

In [ ]:
data_train = utils.join_dataset(X_train, y_train)

Y continuamos con el conjunto de datos de prueba:

In [ ]:
data_test = utils.join_dataset(X_test, y_test)

Para asegurarnos de que se han juntado correctamente, obtenemos una muestra aleatoria del conjunto de datos de entrenamiento:

In [ ]:
data_train.sample(5, random_state=seed)

Y del conjunto de datos de prueba:

In [ ]:
data_test.sample(5, random_state=seed)

## 2. Análisis exploratorio de datos

Antes de continuar con el preprocesamiento, vamos a observar las propiedades del conjunto de datos, examinando sus variables y la interacción entre estas.

### Descripción del conjunto de datos

Primero debemos conocer nuestro problema y para ello tenemos que saber el número de casos y de variables del conjunto:

In [ ]:
data_train.shape

Como se puede observar, el conjunto de datos de entrenamiento está formado por 537 casos y 9 variables (8 predictoras y 1 variable clase).

Para conocer cuál es el tipo de las variables, recurrimos al método `info`:

In [ ]:
data_train.info(memory_usage=False)

Como podemos ver, las variables predictoras son todas númericas, tanto int64 como float64-
Por el contrario, la variable clase (`Outcome`) es categórica (`category`, especificada al cargar el conjunto de datos) y contiene los siguientes estados:

In [ ]:
y_train.cat.categories

Por lo que nuestra variable clase puede tomar dos estados.

### Visualización de las variables

Una vez que hemos visto con más detalle nuestro conjunto de datos de entrenamiento, lo siguiente que haremos será representar y analizar las distribuciones de las variables.
Para ello utilizaremos histogramas para las variables predictoras, ya que son todas numéricas.


In [ ]:
utils.plot_histogram(data_train)

Tal y como se puede observar en la gráfica superior, hay registros en lo que el nivel de glucosa, presión sanguínea, grosor de piel, insulina e índice de masa corporal son 0. Son valores que no tienen sentido, ya que en el caso de no ser erróneos la persona estaría muerta.
Antes de continuar, vamos a visualizar la proporcion de valores perdidos respecto al total de los casos para determinar si es mejor deshacernos directamente de la columna o no:

In [ ]:
data_train[data_train == 0].count(axis=0)/len(data.columns)

Como podemos observar, el mayor porcentaje de casos perdidos lo tiene la insulina (29%). Mantendremos la columna ya que no llega al 30%.

Más adelante procederemos a trataar esos valores perdidos.

Continuamos visualizando las variables categóricas del problema:

In [ ]:
utils.plot_barplot(data_train)

Lo que podemos ver en esta gráfica es que la clase 0 de la variable objetivo del problema tiene más número de casos (casi el doble de casos) que la clase 1, por lo que el problema no está balanceado.

In [ ]:
utils.plot_pairplot(data_train, target="Outcome")

Como podemos observar en el gráfico, no hay ninguna par de variables que produzcan una separación más o menos clara de la variable objetivo.

Podemos analizar variables numéricas (valor medio, mínimo, máximo, etc.) con el método describe (include="number"):


In [ ]:
data_train.describe(include="number")

Y también categóricas (número de casos, estados, etc.) usando igualmente el método describe (include="category"):

In [ ]:
data_train.describe(include="category")

## 3. Preprocesamiento de datos

Vamos a tratar los valores perdidos intrínsecos en las variables vistas anteriormente:

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Value Imputer
imp = SimpleImputer(missing_values=0, strategy='mean')

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('missing', imp ,[False, True, True, True, True, True, True, True]) 
    ], remainder="passthrough")

# Fitting the column transformer
preprocessor = preprocessor.fit(X_train)

#Las columnas a las que no se le aplica la transformación se colocan al final del DataFrame
X_train = pd.DataFrame(preprocessor.transform(X_train), 
                                columns=X_train.columns[1:].append(X_train.columns[:1]))

data_train = utils.join_dataset(X_train, y_train)
X_train.describe()

De esta forma hemos replazado los valores que eran 0 en las variables que no tenía sentido que fueran 0, por la media de los valores que toma esa variable.

## 4. Algoritmos de clasificación y evaluación de modelos

Una vez hemos realizado un análisis exploratorio de los datos y el preprocesamiento de los datos, empezamos con los algoritmos Zero-R y árboles de decisión.

### Algoritmo *Zero-R*

Como sabemos el algoritmo *Zero-R* aprede un clasificador que asigna a los nuevos casos la clase que predomina en el conjunto de entrenamiento, por lo tanto es de esperar que no tenga nada de varianza pero sí mucho sesgo.

Para usar el algoritmo *Zero-R*, recurrimos al estimador `DummyClassifier` de `scikit-learn`:

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")

In [ ]:
zero_r_model.fit(X_train,y_train)

Ahora entreamos y validamos nuestro clasificador, para ello utilizamos la matriz de confusión y  tasa de acierto:

In [ ]:
utils.evaluate(zero_r_model,
               X_train, X_test,
               y_train, y_test)

Tal y como se esperaba, zeroR no obtiene muy buenos resultados, aunque el accuracy no es muy malo debido a que una de las clases era más predominante sobre la otra, por eso no tiene tan mala tasa de acierto. 
Si fuera más equilibrada el accuracy podría ser un 50% o incluso menor, por lo tanto zeroR no obtiene buenos resultados para este conjunto de datos.

### Algoritmo *CART* (*Classification and Regression Trees*): Inducción de árboles de decisión

El siguiente algoritmo que probaremos será un árbol de decisión, sin y con discretización.

Ya que es un método más complejo y competitivo que el Zero-R lo que se espera es conseguir mejores resultados.

In [ ]:
discretizer = KBinsDiscretizer(n_bins=3, strategy="uniform")

In [ ]:
tree_model = DecisionTreeClassifier(random_state=seed)
tree_model.fit(X_train,y_train)

### *Pipeline*

Creamos un *pipeline* para aplicar las transformaciones al conjunto de datos.
Este pipeline estará formado por por `KBinsDiscretizer` + `DecisionTreeClassifier` y así poder ver los resultados obtenidos sin y con discretización.

In [ ]:
discretize_tree_model = make_pipeline(discretizer, tree_model)
discretize_tree_model.fit(X_train,y_train)

In [ ]:
utils.evaluate(tree_model,
               X_train, X_test,
               y_train, y_test)

Y con el conjunto de datos discretizados:

In [ ]:
utils.evaluate(discretize_tree_model,
               X_train, X_test,
               y_train, y_test)

Como podemos observar tampoco se obtienen muy buenos resultados, ya que con discretización obtenemos el mismo accuracy que en ZeroR.
Esto puede deberse a que como vimos en las gráficas anteriores, ninguna de las variables o ningún par e variables realiza una división relativamente clara de la variable objetivo.

 # 2. Carga de datos Wisconsin dataset
## 1. Wisconsin dataset  

Ahora realizaremos el mismo estudio para el conjunto de datos de Wisconsin.

Ahora realizaremos el mismo estudio para el conjunto de datos de Wisconsin.

In [ ]:
filepath = "../input/breast-cancer-wisconsin-data/data.csv"


indexW = "id"
targetW = "diagnosis"

dataW = utils.load_data(filepath, indexW, targetW)
dataW.info()

Como podemos observar en la información del conjunto de datos, tenemos 32 columnas, delas cuales la columna Unnamed:32 no tiene ningún tipo de información, por lo que deberíamos eliminarla antes de continuar con el análisis de los datos.

In [ ]:
dataW=dataW.drop(['Unnamed: 32'], axis=1)

Una vez hemos realizado la carga del conjunto de datos, comprobamos que el proceso se ha realizado correctamente:

In [ ]:
dataW.head(5)

In [ ]:
dataW.sample(5, random_state=seed)

Hemos comprobado que los datos se han cargado correctamente, por lo que vamos a continuar separando el conjunto en dos subconjuntos:

In [ ]:
(X_w, y_w) = utils.divide_dataset(dataW, target="diagnosis")

Y volvemos a comprobar que se haya separado correctamente, primero las variables predictoras y luego la variable clase:

In [ ]:
X_w.sample(5, random_state=seed)

In [ ]:
y_w.sample(5, random_state=seed)

Antes de comenzar con el análisis exploratorio vamos a separar nuestro conjunto de datos en entrenamiento y test.
Aplicamos un holdout estratificado y volvemos a comprar que el conjunto de datos se ha dividido correctamente, tanto entrenamiento como test:

In [ ]:
train_size = 0.7

(X_train_w, X_test_w, y_train_w, y_test_w) = train_test_split(X_w, y_w,
                                                      stratify=y_w,
                                                      random_state=seed,
                                                      train_size=train_size)

In [ ]:
X_train_w.sample(5, random_state=seed)

In [ ]:
X_test_w.sample(5, random_state=seed)

In [ ]:
y_train_w.sample(5, random_state=seed)

In [ ]:
y_test_w.sample(5, random_state=seed)

Volvemos a juntar las variables predictoras con la variable clase en nuestro conjunto de entrenamiento y de test y lo comprobamos:

In [ ]:
data_train_w = utils.join_dataset(X_train_w, y_train_w)

In [ ]:
data_test_w = utils.join_dataset(X_test_w, y_test_w)

In [ ]:
data_train_w.sample(5, random_state=seed)

In [ ]:
data_test_w.sample(5, random_state=seed)

## 2. Análisis exploratorio de datos

Antes de continuar con cualquier operación vamos a examninar el número de casos y de variables del conjunto de entrenamiento:

In [ ]:
data_train_w.shape

Nuestro conjunto de entrenamiento está formado por 398 casos y 31 variables (30 predictoras y 1 variable clase).

El tipo de las variables lo hemos visto anteriormente, además hemos eliminado la variable Unnamed:32 ya que no tenía ningún tipo de información.
Y como hemos visto todas las variables son númericas, excepto la variable diagnosis que es categórica y que contiene los siguientes estados:

In [ ]:
y_train_w.cat.categories

Nuestra variable clase puede tomar dos estados: B y M.

### Visualización de las variables

Después de conocer en detalle el conjunto de entrenamiento, vamos a representar y analizar las distribuciones de las variables y para ello utilizaremos histogramas para las variables numéricas (todas nuestras variables predictoras) y un diagrama de barras para las variables categóricas (nuestra variable clase).

Vamos a visualizar primero un histograma con las variables numéricas:

In [ ]:
utils.plot_histogram(data_train_w)

Como podemos ver en el histograma la mayoría de las variables tienen una distribución en forma acampanada. Aunque si que es cierto que en general se encuentran sesgadas a la derecha, como podemos ver en las variables concave points_mean, symmetry_mean, symmetry worst ...
Y además, podemos destacar que en algunas variables (como concave points_mean, area_worst, smoothness_se, concavity_se, symmetry_se...) es posible que exista algún valor anómalo, ya que se puede ver en la gráfica que hay valores más alejados.

Y con respecto a las variables categóricas, en este caso una única variable:

In [ ]:
utils.plot_barplot(data_train_w)

Podemos observar en el diagrama de barras es que la clase B de la variable objetivo del problea tiene más número de casos, por lo que nuestro problema no está balanceado.

Vamos a estudiar las relaciones que existen entre pares de variables (análisis multivariado), para ver si podemos extraer información:

In [ ]:
utils.plot_pairplot(data_train_w, target="diagnosis")

Con este conjunto de datos encontramos un problema que es la dimensionalidad de los datos.
Visualizando por encima las gráficas y haciendo un poco de zoom en las que vemos que hay más separación para las variables: concave_points_se y fractal_dimension_se principalmente.

También nos podemos apoyar en el método describe para analizar las variables numéricas:

In [ ]:
data_train_w.describe(include="number")

Podemos observar el número de instancias para nuestro conjunto de entrenamiento (398) y también algunos valores (como la media,el mín, el máx...) que nos pueden ayudar a detectar valores erróneos o perdidos, y que en este caso parece que no hay.

Y para la variable categórica:

In [ ]:
data_train_w.describe(include="category")

Como vemos tenemos en nuestro conjunto de entrenamiento 398 casos y al igual que veíamos en el gráfico de barras hay más número de casos de la clase B.

## 3. Algoritmos de clasificación y evaluación de modelos

Una vez hemos realizado un análisis exploratorio de los datos y el preprocesamiento de los datos, empezamos con los algoritmos Zero-R y árboles de decisión.

### Algoritmo *Zero-R*

Usamos el algoritmo *Zero-R* para este conjunto de datos al iugal que para el anterior, y como ya sabemos este algoritmo asigna a los nuevos casos la clase que predomina en el conjunto de entrenamiento. Para nuestro caso la clase que predominaba era la B, por lo que sería de esperar que a los nuevos casos les fuera asignada esta clase.


In [ ]:
zero_r_model_w = DummyClassifier(strategy="most_frequent")
zero_r_model_w.fit(X_train_w,y_train_w)

Ahora entrenamos y validamos nuestro clasificador, para ello utilizamos la matriz de confusión y tasa de acierto:

In [ ]:
utils.evaluate(zero_r_model_w,
               X_train_w, X_test_w,
               y_train_w, y_test_w)

Como esperabamos zeroR no obtiene buenos resultados, aunque si que es cierto queel accuracy no es tan malo, ya que como nos pasaba en el otro conjuto de datos, una clase es más predominante sobre la otra, por eso la mayoría de casos los clasifica correctamente.
Si fuera un conjunto más balanceado en el que no hubiera una clase mucho más predominante, nuestro accuracy bajaría y obtendríamos peores resultados con este algoritmo.

### Algoritmo *CART* (*Classification and Regression Trees*): Inducción de árboles de decisión

El siguiente algoritmo que probaremos será un árbol de decisión, sin y con discretización.

Ya que es un método más complejo y competitivo que el Zero-R lo que se espera es conseguir mejores resultados, además de que como se podía observar en las gráficas anteriores existían variables que realizaban una buena separación de la variable clase.

In [ ]:
discretizer_w = KBinsDiscretizer(n_bins=3, strategy="uniform")

In [ ]:
tree_model_w = DecisionTreeClassifier(random_state=seed)
tree_model_w.fit(X_train_w,y_train_w)

### *Pipeline*

Creamos un *pipeline* para aplicar las transformaciones al conjunto de datos.
Este pipeline estará formado por por `KBinsDiscretizer` + `DecisionTreeClassifier` y así poder ver los resultados obtenidos sin y con discretización.

In [ ]:
discretize_tree_model_w = make_pipeline(discretizer, tree_model_w)
discretize_tree_model_w.fit(X_train_w,y_train_w)

Ahora entrenamos y validamos nuestro clasificador, para ello utilizamos la matriz de confusión y tasa de acierto:

In [ ]:
utils.evaluate(tree_model_w,
               X_train_w, X_test_w,
               y_train_w, y_test_w)

In [ ]:
utils.evaluate(discretize_tree_model_w,
               X_train_w, X_test_w,
               y_train_w, y_test_w)

Como era de esperar, obtenemos unos resultados mucho mejores que con un clasificador ZeroR ya que como hemos indicado existe al menos una variable que es capaz de predecir de forma bastante acertada los nuevos casos.
También hay que comentar que el árbol de decisión que ha sido entrenado con el conjunto de datos discretizado obtiene una mayor tasa de acierto que el no discretizado, debido a que la variable numérica utilizada al transformarla en variable categórica consigue que aumente la tasa de acierto.